In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsOneClassifier
from sklearn import cross_validation

/Users/hknam/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
input_file = './income_data.txt'

In [3]:
X = []
y = []
count_class_1 = 0
count_class_2 = 0
max_datapoints = 25000

In [4]:
with open(input_file, 'r') as f:
    for line in f.readlines():
        if count_class_1 >= max_datapoints and count_class_2 >= max_datapoints:
            break
        if '?' in line:
            continue
        data = line[:-1].split(', ')
        if data[-1] == '<=50K' and count_class_1 < max_datapoints:
            X.append(data)
            count_class_1 += 1
        if data[-1] == '>50K' and count_class_2 < max_datapoints:
            X.append(data)
            count_class_2 += 1            

In [5]:
X = np.array(X)

In [6]:
label_encoder = []
X_encoded = np.empty(X.shape)

In [7]:
for i, item in enumerate(X[0]):
    if item.isdigit():
        X_encoded[:, i] = X[:, i]
    else:
        label_encoder.append(preprocessing.LabelEncoder())
        X_encoded[:, i] = label_encoder[-1].fit_transform(X[:, i])

X = X_encoded[:, :-1].astype(int)
y = X_encoded[:, -1].astype(int)

In [8]:
classifier = OneVsOneClassifier(LinearSVC(random_state = 0))

In [9]:
classifier.fit(X, y)

OneVsOneClassifier(estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
     verbose=0),
          n_jobs=1)

In [10]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size = 0.2, random_state = 5)

In [11]:
classifier = OneVsOneClassifier(LinearSVC(random_state = 0))

In [12]:
classifier.fit(X_train, y_train)

OneVsOneClassifier(estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
     verbose=0),
          n_jobs=1)

In [13]:
y_test_pred = classifier.predict(X_test)

In [14]:
f1 = cross_validation.cross_val_score(classifier, X, y, scoring= 'f1_weighted', cv = 3)

In [15]:
print('F1 score : ' + str(round(100 * f1.mean(), 2)) + '%')

F1 score : 66.82%


In [16]:
input_data = ['37', 'Private', '215646', 'HS-grad', '9', 'Never-married', 'Handlers-cleaners',
              'Not-in-family', 'White', 'Male', '0', '0', '40', 'United-States']

In [17]:
input_data_encoded = [-1] * len(input_data)
count = 0
for i, item in enumerate(input_data):
    if item.isdigit():
        input_data_encoded[i] = int(input_data[i])
    else:
        input_data_encoded[i] = int(label_encoder[count].transform([input_data[i]]))
        count += 1 
input_data_encoded = np.array(input_data_encoded)

In [18]:
predicted_class = classifier.predict([input_data_encoded])
print(label_encoder[-1].inverse_transform(predicted_class)[0])

<=50K
